The data is currently stored as `.xls` files. In this notebook, we will implement some code to manipulate the data as `pandas.Dataframes` and store as more efficient `.parquet` files on disk.

In [1]:
# import any required libraries here
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

First, we need to read the `.xls` files into `pandas.Dataframes`. You can use [pandas.read_excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for this.

In [2]:
# load the building data 
# consider the different number of header rows!
def data_loader(file_name1, file_name2):
    
    # reading files
    df1 = pd.read_excel(file_name1, header=[0, 2, 4])
    df2 = pd.read_excel(file_name2, header=[0, 2, 4])
    
    df1 = df1.drop(0, axis = 0) #dropping the first row
    df1 = df1.reset_index(drop = True) #resetting the index
    df1.rename(columns={"Wärme": "Heat", 'Wasser':'Water', "Elektrizität":"Electricity"}, inplace=True) #renaming the columns
    df1.columns = [f'{i}_{j[-8:]}_{k}' for i, j, k in df1.columns] #merging the column names into a unique column name
    df1 = df1.rename({'Energieart_hreibung_OBIS Bezeichnung' : 'Time'}, axis = 1) #renaming the column to time
   
    df2 = df2.drop(0, axis = 0) #dropping the first row
    df2 = df2.reset_index(drop = True) #resetting the index
    df2.rename(columns={"Wärme": "Heat", 'Wasser':'Water', "Elektrizität":"Electricity"}, inplace=True) #renaming the columns
    df2.columns = [f'{i}_{j[-8:]}_{k}' for i, j, k in df2.columns] #merging the column names into a unique column name
    df2 = df2.rename({'Energieart_hreibung_OBIS Bezeichnung' : 'Time'}, axis = 1)#renaming the column to time
    
    concat_df = pd.concat([df1, df2]) # concating data frames
    return concat_df

In [3]:

chemie_df = data_loader(file_name1 = 'Chemie.xls', file_name2 = 'Chemie_01_26-07_19.xls')
gross_df = data_loader(file_name1 = 'Großtagespflege.xls', file_name2 = 'Grosstagespflege_04_05-07_19.xls')
hg_2_df  = data_loader(file_name1 = 'HG II.xls', file_name2 = 'HGII_01_26-07_19.xls')
kita_hokida_df = data_loader(file_name1 = 'Kita Hokido.xls', file_name2 = 'Kita Hokido_05_22_20-07_19_22.xls')
oh12_df = data_loader(file_name1 = 'OH12.xls', file_name2 = 'OH12_01_26-07_19.xls')
oh14_df = data_loader(file_name1 = 'OH14.xls', file_name2 = 'OH14_01_26-07_19.xls')


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Next, we need to implement a function that takes a `pandas.Dataframe` and a path string as an input and writes the data to disk as a `parquet` file. You can use the [PyArrow library](https://arrow.apache.org/docs/python/parquet.html) for this: 

In [4]:
def write_as_parquet(df, path):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, path)
    # constructing a table from pandas data frame and passing it to write_table

In [5]:
write_as_parquet(chemie_df, path = 'chemie.parquet')
write_as_parquet(gross_df, path = 'gross.parquet')
write_as_parquet(hg_2_df, path = 'hg_2.parquet')
write_as_parquet(kita_hokida_df, path = 'kita_hokida.parquet')
write_as_parquet(oh12_df, path = 'oh12.parquet')
write_as_parquet(oh14_df, path = 'oh14.parquet')

Now we need the opposite functionality: a function that reads data from a `.parquet` file on disk and returns it as a `pandas.Dataframe`. Implement this function such that it can take a list of names of column to load as an _optional_ parameter. 

In [6]:
def load_to_pandas(path):
    # reading the parquet file and converting it into pandas dataframe
    df = pq.read_pandas(path).to_pandas()
    return df

In [8]:
load_to_pandas(path = 'gross.parquet')

,Time,Heat_12 01 01_Wärmeenergie Tarif 1,Heat_12 01 01_Durchfluss,Heat_12 01 01_Volumen,Heat_12 01 01_Vorlauftemperatur,Heat_12 01 01_Rücklauftemperatur,Heat_12 01 01_Temperaturdifferenz,Heat_12 01 01_Wärmeleistung,Heat_12 02 01_Wärmeenergie Tarif 1,Heat_12 02 01_Durchfluss,...,Electricity_72 01 03_BV+ Arbeit Tarif 1,Electricity_72 01 03_BV- Arbeit tariflos,Electricity_72 01 03_BV- Arbeit Tarif 1,Electricity_72 01 04_P Summe,Electricity_72 01 04_WV+ Arbeit tariflos,Electricity_72 01 04_WV+ Arbeit Tarif 1,Electricity_72 01 05_WV+ Arbeit tariflos,Electricity_72 01 05_WV+ Arbeit Tarif 1,Electricity_72 01 06_WV- Arbeit tariflos,Electricity_72 01 06_WV- Arbeit Tarif 1
0,2022-04-05 02:30:00,3788.0,0.0,516.666,20.7,20.6,0.0,0.0,4377.0,0.109,...,NaN,NaN,NaN,NaN,270.5456,270.5456,387.5243,387.5243,6073.0165,6073.0165
1,2022-04-05 02:15:00,3788.0,0.0,516.666,20.8,20.8,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5398,270.5398,387.5117,387.5117,6073.0165,6073.0165
2,2022-04-05 02:00:00,3788.0,0.0,516.666,20.9,21.0,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5340,270.5340,387.4991,387.4991,6073.0165,6073.0165
3,2022-04-05 01:45:00,3788.0,0.0,516.666,21.1,21.1,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5283,270.5283,387.4866,387.4866,6073.0165,6073.0165
4,2022-04-05 01:30:00,3788.0,0.0,516.666,21.2,21.1,0.1,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5225,270.5225,387.4740,387.4740,6073.0165,6073.0165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10086,2022-04-05 01:00:00,3788.0,0.0,516.666,21.5,21.5,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5110,270.5110,387.4491,387.4491,NaN,NaN
10087,2022-04-05 00:45:00,3788.0,0.0,516.666,21.7,21.7,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.5053,270.5053,387.4366,387.4366,NaN,NaN
10088,2022-04-05 00:30:00,3788.0,0.0,516.666,21.9,21.9,0.0,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.4995,270.4995,387.4240,387.4240,NaN,NaN
10089,2022-04-05 00:15:00,3788.0,0.0,516.666,22.0,21.9,0.1,0.0,4377.0,0.000,...,NaN,NaN,NaN,NaN,270.4938,270.4938,387.4116,387.4116,NaN,NaN


Great! We can now store data more efficiently on disk and know how to load it again. Store all the data we have as one `.parquet` file per building.